In [23]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
tf.set_random_seed(777)
tf.enable_eager_execution()

x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)#(8, 4)
y_data = np.asarray(y_data, dtype=np.float32)#(8, 3)

# dataset을 선언한다.
# dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
# dataset = dataset.repeat().batch(2)
nb_classes = 3 #class의 갯수
print('x_data shape: {0}'.format(x_data.shape))
print('y_data shape: {0}'.format(y_data.shape))

#Weight and bias setting
W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')#(4,3)
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')#(3,)
variables = [W, b]
print('===== W, b =====')
print('@@@@ W: {0}, \n@@@@ b: {1}'.format(W, b))
print('================\n')


# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis(X):
  return tf.nn.softmax(tf.matmul(X, W) + b)# X(8,4) * W(4,3) : (8,3)
print('===== hypothesis =====')
print('hypothesis(x_data): {0}'.format(hypothesis(x_data)))
print('======================\n')

sample_db = [[8, 2, 1, 4]]#(1,4)
sample_db = np.asarray(sample_db, dtype=np.float32)
print('===== hypothesis =====')
print('hypothesis(sample_db): {0}'.format(hypothesis(sample_db)))#X(1,4) * W(4,3): (1,3)
print('======================\n')

def cost_fn(X, Y):
  logits = hypothesis(X)
  cost = -tf.reduce_sum(Y * tf.log(logits), axis=1)# -1* sum(Y * log(XW+b)), sum: 차원의 총합을 계산
  cost_mean = tf.reduce_mean(cost)
  return cost_mean

print('cost_fn: {0}'.format(cost_fn(x_data, y_data)))

x = tf.constant(3.0)
with tf.GradientTape() as g:
  g.watch(x)
  y = x*x
  dy_dx = g.gradient(y, x)#Will compute to 6.0
  print('dy_dx: {0}'.format(dy_dx))

def grad_fn(X, Y):
  with tf.GradientTape() as tape:
    loss = cost_fn(X, Y)
    grads = tape.gradient(loss, variables)
    return grads
print('grad_fn: {0}'.format(grad_fn(x_data, y_data)))

def fit(X, Y, epochs=2000, verbose=100):
  optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
  
  for i in range(epochs):
    grads = grad_fn(X,Y)
    optimizer.apply_gradients(zip(grads, variables))
    if (i == 0) | ((i+1)%verbose==0):
      print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))

fit(x_data, y_data)

x_data shape: (8, 4)
y_data shape: (8, 3)
===== W, b =====
@@@@ W: <tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.7706481 ,  0.37335402, -0.05576323],
       [ 0.00358377, -0.5898363 ,  1.5702795 ],
       [ 0.2460895 , -0.09918973,  1.4418385 ],
       [ 0.3200988 ,  0.526784  , -0.7703731 ]], dtype=float32)>, 
@@@@ b: <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-1.3080608 , -0.13253094,  0.5513761 ], dtype=float32)>

===== hypothesis =====
hypothesis(x_data): [[1.3657196e-02 7.9016229e-03 9.7844124e-01]
 [3.9259814e-02 1.7034756e-02 9.4370544e-01]
 [3.8038522e-01 1.6772322e-01 4.5189154e-01]
 [3.2339045e-01 5.9075940e-02 6.1753362e-01]
 [3.6299741e-06 6.2072729e-08 9.9999630e-01]
 [2.6252046e-02 1.0727973e-02 9.6301997e-01]
 [1.5652511e-05 4.2180275e-07 9.9998391e-01]
 [2.9407772e-06 3.8113402e-08 9.9999702e-01]]

===== hypothesis =====
hypothesis(sample_db): [[0.9302204  0.06200534 0.00777428]]

cost_fn: 6.079319477081299
dy_dx: 6.0
grad_fn: [